In [3]:
from gensim.models import KeyedVectors

# 加载预训练的词向量模型
model = KeyedVectors.load_word2vec_format(r'tencent-ailab-embedding-zh-d100-v0.2.0-s.txt', binary=False)

# 定义歧义词汇和测试用例
word = '苹果'
meanings = {
    '水果': ['这个苹果真好吃，营养价值高。', '今天买了一箱苹果。'],
    '电子品牌': ['苹果手机配置不高，但整体性能优化的很好。', '我想买一台苹果电脑。'],
    '电影名称': ['苹果上映了，听说非常精彩。', '你看过苹果这部电影吗？']
}

# 定义一个函数来计算每个义项和测试用例之间的相似度，并返回最佳的义项
def disambiguate(word, meanings, model):
    best_meaning = None
    max_similarity = -1

    # 遍历每个义项和测试用例
    for meaning, examples in meanings.items():
        for example in examples:
            # 对每个测试用例，计算其与当前义项的相似度
            similarity = model.n_similarity(word, example)

            # 如果相似度最高，则更新最佳义项和相似度
            if similarity > max_similarity:
                max_similarity = similarity
                best_meaning = meaning

    # 返回最佳义项
    return best_meaning

# 调用函数进行词义消歧，并打印结果
result = disambiguate(word, meanings, model)
print(result)


水果


In [5]:
import jieba
from gensim.models import KeyedVectors

# 加载预训练的词向量模型
model = KeyedVectors.load_word2vec_format(r'tencent-ailab-embedding-zh-d100-v0.2.0-s.txt', binary=False)

# 定义一个函数来计算每个义项和测试句子之间的相似度，并返回最佳的义项
def disambiguate(word, meanings, model, sentence):
    best_meaning = None
    max_similarity = -1

    # 对句子进行分词，并将单词拆分成字
    words = list(jieba.cut(sentence))
    chars = []
    for word in words:
        chars.extend(list(word))

    # 遍历每个义项
    for meaning, examples in meanings.items():
        # 对每个测试用例，计算其与当前义项的相似度
        similarity = 0
        for example in examples:
            # 将例句拆分成字
            example_chars = list(example)
            # 对于每个字，计算它与测试句子中每个字的余弦相似度，并取平均值作为句子与例句的相似度
            char_similarity = 0
            for char in chars:
                if char in model.key_to_index and char in example_chars:
                    char_similarity += model.similarity(char, char)
            if len(chars) > 0:
                char_similarity /= len(chars)
            similarity += char_similarity
        similarity /= len(examples)

        # 如果相似度最高，则更新最佳义项和相似度
        if similarity > max_similarity:
            max_similarity = similarity
            best_meaning = meaning

    # 返回最佳义项
    return best_meaning

# 定义歧义词汇和测试句子
word_1 = '苹果'
meanings_1 = {
    '水果': ['这个苹果真好吃，营养价值高。', '今天买了一箱苹果。'],
    '电子品牌': ['苹果手机配置不高，但整体性能优化的很好。', '我想买一台苹果电脑。'],
    '电影名称': ['苹果上映了，听说非常精彩。', '你看过苹果这部电影吗？']
}
sentences_1 = [
    '我喜欢吃苹果。',
    '我要买一个苹果手机。',
    '昨天看了一部叫做苹果的电影。'
]

word_2 = '水分'
meanings_2 = {
    '物体内所含的水': ['植物是靠它的根从土壤中吸收水分。'],
    '比喻某一情况中夹杂的不真实的成分': ['他说的话里有很大的水分。'],
    '比喻某一情况中的冗余、浪费部分': ['这个计划里有很多水分，需要重新制定。']
}
sentences_2 = [
    '这个新闻里的水分很大。',
    '他的话有点水分，不太可信。'
]

# 遍历每个测试句子，并输出最佳义项
print(word_1)
for sentence in sentences_1:
    meaning = disambiguate(word_1, meanings_1, model, sentence)
    print(f'"{sentence}" 的最佳义项为：{meaning}')
print("------------------------------------")
print(word_2)
for sentence in sentences_2:
    meaning = disambiguate(word_2, meanings_2, model, sentence)
    print(f'"{sentence}" 的最佳义项为：{meaning}')

苹果
"我喜欢吃苹果。" 的最佳义项为：电子品牌
"我要买一个苹果手机。" 的最佳义项为：电子品牌
"昨天看了一部叫做苹果的电影。" 的最佳义项为：电影名称
------------------------------------
水分
"这个新闻里的水分很大。" 的最佳义项为：比喻某一情况中的冗余、浪费部分
"他的话有点水分，不太可信。" 的最佳义项为：比喻某一情况中夹杂的不真实的成分
